In [22]:
# import dataset of weather events
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/WeatherEvents.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
df.info

<bound method DataFrame.info of            EventId  Type  Severity       StartTime(UTC)         EndTime(UTC)  \
0              W-1  Snow     Light  2016-01-06 23:14:00  2016-01-07 00:34:00   
1              W-2  Snow     Light  2016-01-07 04:14:00  2016-01-07 04:54:00   
2              W-3  Snow     Light  2016-01-07 05:54:00  2016-01-07 15:34:00   
3              W-4  Snow     Light  2016-01-08 05:34:00  2016-01-08 05:54:00   
4              W-5  Snow     Light  2016-01-08 13:54:00  2016-01-08 15:54:00   
...            ...   ...       ...                  ...                  ...   
8627176  W-9091890  Snow     Light  2022-12-21 23:00:00  2022-12-21 23:42:00   
8627177  W-9091891  Snow  Moderate  2022-12-21 23:42:00  2022-12-21 23:53:00   
8627178  W-9091892  Cold    Severe  2022-12-21 23:53:00  2022-12-24 02:53:00   
8627179  W-9091893  Cold    Severe  2022-12-24 03:53:00  2022-12-24 07:53:00   
8627180  W-9091894  Cold    Severe  2022-12-24 09:53:00  2022-12-24 11:53:00   

       

In [24]:
# adjust the format of datetime in weather.csv file
df['StartTime(UTC)'] = pd.to_datetime(df['StartTime(UTC)'])
df['Date'] = df['StartTime(UTC)'].dt.strftime('%m-%Y')

In [25]:
# extract weather data of selected cities that both existed in weatherevents dataset and yelp dataset
city_list = ['Indianapolis','Nashville','Philadelphia','Tampa','Tucson','New Orleans','Reno','Saint Louis']
city_data = df[df['City'].isin(city_list)]

In [26]:
city_data.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,Date
491694,W-519112,Snow,Light,2016-01-01 08:15:00,2016-01-01 08:55:00,0.00,US/Pacific,KRTS,39.6682,-119.8764,Reno,Washoe,NV,89506.0,01-2016
491695,W-519113,Snow,Light,2016-01-05 00:15:00,2016-01-05 00:35:00,0.00,US/Pacific,KRTS,39.6682,-119.8764,Reno,Washoe,NV,89506.0,01-2016
491696,W-519114,Rain,Light,2016-01-05 00:35:00,2016-01-05 00:55:00,0.00,US/Pacific,KRTS,39.6682,-119.8764,Reno,Washoe,NV,89506.0,01-2016
491697,W-519115,Snow,Light,2016-01-05 17:55:00,2016-01-05 19:55:00,0.02,US/Pacific,KRTS,39.6682,-119.8764,Reno,Washoe,NV,89506.0,01-2016
491698,W-519116,Snow,Moderate,2016-01-05 19:55:00,2016-01-05 20:35:00,0.08,US/Pacific,KRTS,39.6682,-119.8764,Reno,Washoe,NV,89506.0,01-2016


In [27]:
# calculate the sum of precipitation in certain city at given year and month
precipitation_sum = city_data.groupby(['City', 'Date'])['Precipitation(in)'].sum().reset_index()
# calculate the sum of weather events in certain city at given year and month
events_sum = city_data.groupby(['City', 'Date', 'Type', 'Severity']).size().reset_index(name='Count')

print(precipitation_sum)
print(events_sum)

             City     Date  Precipitation(in)
0    Indianapolis  01-2016               2.87
1    Indianapolis  01-2017              14.66
2    Indianapolis  01-2018               5.97
3    Indianapolis  01-2019               7.66
4    Indianapolis  01-2020              15.65
..            ...      ...                ...
667        Tucson  12-2018               4.16
668        Tucson  12-2019               2.36
669        Tucson  12-2020               0.58
670        Tucson  12-2021               2.68
671        Tucson  12-2022               2.42

[672 rows x 3 columns]
              City     Date  Type  Severity  Count
0     Indianapolis  01-2016   Fog  Moderate      6
1     Indianapolis  01-2016   Fog    Severe     11
2     Indianapolis  01-2016  Rain     Light     30
3     Indianapolis  01-2016  Rain  Moderate      1
4     Indianapolis  01-2016  Snow     Heavy      2
...            ...      ...   ...       ...    ...
3952        Tucson  12-2022  Cold    Severe      2
3953        Tucs

In [28]:
# export outputs to Google Drive
from google.colab import drive
precipitation_sum.to_csv('/content/drive/MyDrive/precipitation_sum.csv', index=False)
events_sum.to_csv('/content/drive/MyDrive/events_sum.csv', index=False)


In [29]:
# merge weather data with yelp data
# the extracted yelp data are filtered in mysql workbench
yelp = pd.read_csv('/content/drive/My Drive/yelp.csv')
# match weather data and yelp data by City and Date
merged_precipitation = pd.merge(precipitation_sum, yelp, left_on=['City', 'Date'], right_on=['city', 'mon'], how='inner')
merged_events = pd.merge(events_sum, yelp, left_on=['City', 'Date'], right_on=['city', 'mon'], how='inner')


In [31]:
print(merged_precipitation)
print(merged_events)

             City     Date  Precipitation(in)          city      mon  \
0    Indianapolis  01-2019               7.66  Indianapolis  01-2019   
1    Indianapolis  01-2020              15.65  Indianapolis  01-2020   
2    Indianapolis  01-2021               5.95  Indianapolis  01-2021   
3    Indianapolis  01-2022               1.78  Indianapolis  01-2022   
4    Indianapolis  02-2019              17.27  Indianapolis  02-2019   
..            ...      ...                ...           ...      ...   
291        Tucson  11-2020               0.06        Tucson  11-2020   
292        Tucson  11-2021               0.04        Tucson  11-2021   
293        Tucson  12-2019               2.36        Tucson  12-2019   
294        Tucson  12-2020               0.58        Tucson  12-2020   
295        Tucson  12-2021               2.68        Tucson  12-2021   

     avg_star  avg_review_count  restaurant_number  
0    3.900719          356.9376                902  
1    3.922679          311.52

In [32]:
# export merged file
merged_precipitation.to_csv('/content/drive/MyDrive/merged_precipitation.csv', index=False)
merged_events.to_csv('/content/drive/MyDrive/merged_events.csv', index=False)
